In [ ]:
# number og edges = number of vertices-1
# all nodes are reachable , ie no splitted components

# if a normal tree is converted to spanning tree then numbe of possible spanning trees  = 
# ((E)C(E-1)) - (number of loops)